# receive_message

> Receiving a CAN message from a bus
> using the python-can library
> testing on vcan interface

In [1]:
#| default_exp receive_message

In [2]:
#| hide
from __future__ import annotations
from nbdev.showdoc import *
from fastcore.test import *
from pprint import pprint

In [3]:
#| export
import subprocess
import os
import signal
import time
import sys
import io
# from multiprocessing import Process, Event
# from multiprocessing import synchronize, Manager
# from multiprocessing.managers import DictProxy
from typing import Optional
from datetime import datetime
import json

In [4]:
#| export
import can
from can.interface import Bus
from can import Message 
import cantools
from cantools.database import Message as MessageTpl

In [5]:
db = cantools.database.load_file('../res/motohawk_new.dbc')
db.messages
example_message: MessageTpl = db.get_message_by_name('ExampleMessage')
pprint(example_message.signals)
# pprint(example_message.__dict__)
example_message.frame_id

[message('ExampleMessage', 0x1f0, False, 8, {None: 'Example message used as template in MotoHawk models.'}),
 message('NewMessage', 0x254, False, 8, {None: 'self made message'})]

[signal('Enable', 7, 1, 'big_endian', False, None, 1, 0, None, None, '-', False, None, {0: 'Disabled', 1: 'Enabled'}, None, None),
 signal('AverageRadius', 6, 6, 'big_endian', False, None, 0.1, 0, 0, 5, 'm', False, None, None, None, None),
 signal('Temperature', 0, 12, 'big_endian', True, None, 0.01, 250, 229.52, 270.47, 'degK', False, None, None, None, None)]


496

In [6]:
!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo modprobe vcan
!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link add dev vcan0 type vcan
# sshpass -v -p asdf sudo ip link add dev vcan0 type vcan
!ip link show vcan0
# !gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set vcan0 type vcan bitrate 500000  # vcan Does not SUPPORT set bitrate on command line!
# !sshpass -p asdf sudo ip link add dev vcan0 type vcan
!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set up vcan0
# !sshpass -v -p asdf sudo ip link set up vcan0

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.


SSHPASS: read: 

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 

14: vcan0: <NOARP> mtu 72 qdisc noop state DOWN mode DEFAULT group default qlen 1000
    link/can 
SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 



In [7]:
data_dict ={'Temperature': 250.1, 'AverageRadius': 3.2, 'Enable': 1}
data_json_bytes = json.dumps(data_dict).encode('utf-8')
data_json_bytes
json.loads(data_json_bytes.decode())

can_data = example_message.encode({'Temperature': 250.1, 'AverageRadius': 3.2, 'Enable': 1})
example_message.decode(can_data)

b'{"Temperature": 250.1, "AverageRadius": 3.2, "Enable": 1}'

{'Temperature': 250.1, 'AverageRadius': 3.2, 'Enable': 1}

{'Enable': 'Enabled', 'AverageRadius': 3.2, 'Temperature': 250.1}

In [8]:
f = io.BytesIO(data_json_bytes)

In [9]:
bus = Bus(bustype='socketcan', channel='vcan0', bitrate=250000)
data = example_message.encode({'Temperature': 250.1, 'AverageRadius': 3.2, 'Enable': 1})
message_to_send = Message(arbitration_id=example_message.frame_id, data=data, is_extended_id=False) 	
# can_bus.send(message)

In [10]:
manager = Manager()
message_proxy = manager.dict()

NameError: name 'Manager' is not defined

In [ ]:
#| export
def receive_message(message_proxy: DictProxy,bus: Bus)->None:
	print('waiting for message')
	msg:Message = bus.recv()
	print('message received')
	message_proxy['timestamp'] = msg.timestamp
	message_proxy['arbitration_id'] = msg.arbitration_id
	message_proxy['data']=msg.data

In [ ]:

os.getcwd()

'/home/n/devel/anting_remote/eos/tspace/can/nbs'

In [ ]:
proc = subprocess.Popen(['python', 'message_send.py'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, stdin=subprocess.PIPE)
print(f'PARENT: {proc.pid} before signaling child')
sys.stdout.flush()
# time.sleep(1)


PARENT: 1871341 before signaling child


In [ ]:

receive_message(message_proxy, bus)

waiting for message


In [ ]:
print(f'PARENT: {proc.pid} signaling child')
sys.stdout.flush()
os.kill(proc.pid, signal.SIGUSR1)
stdout_value = proc.communicate()[0].decode('utf-8')
print(f'stdout: {repr(stdout_value)}')

PARENT: 803919 signaling child
stdout: 'Signal received after 0.007 seconds\n'


In [ ]:
datetime.fromtimestamp(message_proxy['timestamp']),db.decode_message(message_proxy['arbitration_id'],message_proxy['data'])

(datetime.datetime(2024, 2, 6, 17, 39, 15, 270933),
 {'Enable': 'Enabled', 'AverageRadius': 3.2, 'Temperature': 250.1})

In [ ]:
# close and remove vcan0
# !sshpass -v -p  asdf sudo ip link delete vcan0 

!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set down vcan0

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.


SSHPASS: read: 



In [ ]:

!gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link delete vcan0 

SSHPASS: searching for password prompt using match "assword"
SSHPASS: read: [sudo] password for n: 
SSHPASS: detected prompt. Sending password.
SSHPASS: read: 

